# CSC 522 Project 
_by Team 21: Raphael Phillips (rjphill4), Sogolsadat Mansouri (smansou2), Rithik Jain (rjain25), and Neeloy Gomes (ntgomes)_

This project will involve developing a machine learning model using supervised learning to predict future exchange rates for a specific currency pair, based on historical exchange rate data and other relevant features.


In [29]:
%run pre_processing.py

usdinr_df = pd.read_csv('../data/USDINRX.csv')
forex_df = pd.read_csv('../data/forex.csv')

print('Instantiated the input CSVs as dataframes')

cleaned_usdinr_df = pre_p_usdinr(usdinr_df)
cleaned_forex_df = pre_p_forex(forex_df)

print('Cleaned ' + str(len(forex_df.index) - len(cleaned_forex_df.index)) + ' rows from data/forex.csv')
print('Cleaned ' + str(len(usdinr_df.index) - len(cleaned_usdinr_df.index)) + ' rows from data/USDINRX.csv')

Instantiated the input CSVs as dataframes
Cleaned 1448456 rows from data/forex.csv
Cleaned 52 rows from data/USDINRX.csv


In [30]:
merged_df = merge(cleaned_forex_df, cleaned_usdinr_df)
print('Merged the two datasets')
merged_df

Merged the two datasets


,date,open,high,low,close
0,12387,45.709000,45.728001,45.615002,45.709999
1,12388,45.709000,45.719002,45.560001,45.629002
2,12389,45.632000,45.655998,45.474998,45.549999
3,12390,45.548000,45.612999,45.519001,45.548000
4,12391,45.549999,45.566002,45.449001,45.449001
...,...,...,...,...,...
9172,18890,73.696198,73.899597,73.607002,73.696098
9173,18891,73.659302,73.790100,73.549004,73.658997
9174,18892,73.795097,73.941498,73.650002,73.792702
9175,18893,73.875000,73.925797,73.588997,73.874901


In [31]:
merged_x_train, merged_x_test, merged_y_train, merged_y_test = split(merged_df)
print('Merged and did the train/test split of the datasets')

Merged and did the train/test split of the datasets


In [37]:
%run lstm_rnn.py
%run rand_network_ensemble.py

lstm_args = { 
    "input_shape": (len(merged_x_train.to_numpy()), len(merged_x_train.to_numpy()[0])), 
    "lstm_units": 32, 
    "dense_units": 16, 
    "output_shape": (1,) 
}
ensembler = RandomNetworkEnsemble(num_networks = 10, base_nn_model = CustomLSTM, model_args = lstm_args, random_seed = 522)

ensembler.train(merged_x_train.to_numpy(), merged_y_train.to_numpy())
predicted_target = ensembler.predict(merged_x_test.to_numpy())

Epoch 1/10
Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 3237.7278
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 3229.9966
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 3221.7520
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 3213.0498
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 3204.2971
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 3194.2205
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 3183.4172
Epoch 8/10
1/1 [==============================] - 2s 2s/step - loss: 3173.5510
Epoch 9/10
1/1 [==============================] - 2s 2s/step - loss: 3161.8787
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 3149.5027
Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 3215.9094
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 3205.8025
Epoch 3/10
1/1 [========================

In [38]:
print('RMSE of tuned model\'s predictions: ' + str(ensembler.calculate_rmse_of_predicted(merged_y_test.to_numpy())))

predicted_target

RMSE of tuned model's predictions: 56.53793166476693


array([0.05988596, 0.1086991 , 0.17072818, ..., 0.8531227 , 0.8531227 ,
       0.8531227 ], dtype=float32)